# Captain's Log

In [79]:
# Setup

# pip3 install urllib3 # Might not have worked. Hard to say
# pip install urllib3
# pip3 install requests



In [78]:
#
#   MODEL - data structures
#

# Score data structure
class Score:
    def __init__(self, total, item):
        self.total = total
        self.item = item
    def __repr__(self):
        return repr((self.total, self.item))

# SuperPoint data structure
class SuperPoint:
    def __init__(self, item):
        self.items = []
        self.items.append(item)
    def __repr__(self):
        return repr((self.items))



In [80]:
#
# UTILITY methods
#
from math import sqrt

# Get the distance between two points
def getDistance(loc1, loc2):
    dist = sqrt( (loc2['lon'] - loc1['lon'])**2 + (loc2['lat'] - loc1['lat'])**2 )
    return dist

# Are two points close enough
def closeEnough(loc1, loc2, limitDistance):
    thisDistance = getDistance(loc1, loc2)
    return thisDistance <= limitDistance

# Count the number of points within a distance of a point
def getScore(point, allPoints, limitDistance):
    result = 0
    for other in allPoints:
        if closeEnough(point, other, limitDistance):
            result += 1
    return result

# Convert anything to json
import json
def getJson(o):
    json_string = json.dumps(topNList)    
    return json_string



In [81]:
#
# SETUP
#

import json, urllib.request, requests
from collections import namedtuple

def getLocations(url):
    r = requests.get(url)
    j = json.loads(r.text)
    return j

sample_locations = getLocations('https://geo-pg.captainjustin.space/transactions_rs_time')

# Decide on accuracy
arbitaryDistanceLimit = 0.0001




In [83]:
### Let's score each point by how many other points are within `arbitaryDistanceLimit`


# We'll keep scores here in an array here
scores = [] 


# Get the score for each location
for l in sample_locations:
    total = getScore(l, sample_locations, arbitaryDistanceLimit)
    scores.append(Score(total, l))

sorted_scores = sorted(scores, key=lambda score: score.total, reverse=True)

print("Scoring is done and sorted")
print(len(sorted_scores))


Scoring is done and sorted
1036


In [84]:
# Create the first SuperPoint

def createSuperPoint(scoreList):
    point = scoreList[0].item
    sp = SuperPoint(point)
    scoreList.pop(0)
    i = 0
    while i < len(scoreList):
        other = scoreList[i].item
        if closeEnough(point, other, arbitaryDistanceLimit):
            sp.items.append(other)
            scoreList.pop(i)
            i -= 1
        i += 1
    return sp

# Lets keep out super points together
superPoints = []

sp1 = createSuperPoint(sorted_scores)
print(len(sp1.items))
sp2 = createSuperPoint(sorted_scores)
print(len(sp2.items))




339
5
